# Azure Cognitive Services API

-----

## Face API

The [Microsoft Face API](https://docs.microsoft.com/en-us/azure/cognitive-services/face/overview) is a cloud-based service that provides the most advanced face algorithms. Face API has two main functions: face detection with attributes and face recognition.

For this case study, we'll only be using the _Face Detection with Attributes_ functionality, but I encourage you to investigate the APIs and tutorials for _Face Detection_, as well. You can find them [here](https://docs.microsoft.com/en-us/azure/cognitive-services/face/overview).

-----

![No, not that kind of bacon. But now I'm hungry...](https://images7.alphacoders.com/549/549097.jpg)

# Kevin Bacon Numbers

## _Guessing Kevin Bacon's Age with the Azure Cognitive Services Face API_

-----

### Import Required Libraries

The libraries below allow us to make requests to URLs, and to parse those results. For more information about the libraries we are importing into our project, take a look at the following links:

In [7]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, requests, json, cv2
import numpy as np

### User-Defined Subscription Key 

In [8]:
subscription_key = '' # Change to be your API key.
uri_base = 'https://westcentralus.api.cognitive.microsoft.com' # Change to match your region.

### Request Headers

In [9]:
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': subscription_key,
}

### Request Parameters

There are, to date, three optional request parameters that you can return through the Azure Cognitive Services Face API. The first two (`returnFaceID` and `returnFaceLandmarks`) are **Boolean**, which means that you can select either *True* or *False*. The final parameter, `returnFaceAttributes`, provides the most information.

* `returnFaceID`: Returns faceIds of the detected faces or not. The default value is true.

* `returnFaceLandmarks`: Returns face landmarks of the detected faces or not. The default value is false.

* `returnFaceAttributes`: Analyzes and returns the one or more specified face attributes in the comma-separated string. 

Supported face attributes include age, gender, headPose, smile, facialHair, glasses, emotion, hair, makeup, occlusion, accessories, blur, exposure and noise. Note that each face attribute analysis has additional computational and time cost. For our case study today, we're gonna use 'em all.

* **age**: an age number in years.
* **gender**: male or female.
* **smile**: smile intensity, a number between [0,1]
* **facialHair**: consists of lengths of three facial hair areas: moustache, beard and sideburns.
* **headPose**: 3-D roll/yew/pitch angles for face direction. Pitch value is a reserved field and will always return 0.
* **glasses**: glasses type. Possible values are 'NoGlasses', 'ReadingGlasses', 'Sunglasses', 'SwimmingGoggles'.
* **emotion**: emotions intensity expressed by the face, incluing anger, contempt, disgust, fear, happiness, neutral, sadness and surprise.
* **hair**: return face features indicating whether the hair is visible, bald or not also including hair color if available.
* **makeup**: whether face area (eye, lip) is made-up or not.
* **accessory**: accessory types on face or around face area, including headwear, glasses and mask. Empty array means no accessories detected.
* **occlusion**: whether face area (forehead, eye, mouth) is occluded or not.
* **blur**: blur level of the face. The level include `Low`, `Medium` and `High`. Larger value means more blury the face is.
* **exposure**: exposure level of the face. The level include `GoodExposure`, `OverExposure` and `UnderExposure`.
* **noise**: noise level of the face. The lev}el include `Low`, `Medium` and `High`. Larger value means more noisy the face is.

In [10]:
# Request parameters.
params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

### Request Body

The **Request Body** is where you input the URL for each picture you'd like to analyze. 

Some important things to remember, regarding constraints for the image files:

* JPEG, PNG, GIF (the first frame), and BMP are supported. 
* Image file sizes should be larger than or equal to 1KB but no larger than 4MB.
* The detectable face size is between 36x36 to 4096x4096 pixels. The faces out of this range will not be detected.
* A maximum of 64 faces could be returned for an image. The returned faces are ranked by face rectangle size in descending order.

Some faces may not be detected for technical challenges, e.g. very large face angles (head-pose) or large occlusion. Frontal and near-frontal faces have the best results.

Today, we'll be looking at Kevin Bacon in _Footloose_, 1984 (age 26):

![](https://peopledotcom.files.wordpress.com/2016/08/kevin-bacon-435-17.jpg?w=435)

In _Tremors_, circa 1990 (age 32):

![](http://nerdist.com/wp-content/uploads/2017/06/Kevin-Bacon-in-Tremors.jpg)

In _Wild Things_, circa 1998 (age 40):

![](http://1.bp.blogspot.com/-tSvculYaG4s/VFmUxCD4MJI/AAAAAAAAIAA/7ts97t-ha_c/s1600/movie-pic_002_446_kevin-bacon_wild-things.png)

In _Frost/Nixon_, circa 2008 (age 50):

![](https://images.fandango.com/ImageRenderer/400/0/redesign/static/img/default_poster.png/0/images/masterrepository/fandango/113324/frostnixon-pm-kevinbacon-7.jpg)

And just recently in 2017 (age 59):

![](http://www3.pictures.zimbio.com/gi/Kevin+Bacon+89W-EsX1Suim.jpg)

In [21]:
body = {'url': 'https://peopledotcom.files.wordpress.com/2016/08/kevin-bacon-435-17.jpg?w=435'}

# We will uncomment out the pictures as we cycle through them.

#body = {'url': 'http://nerdist.com/wp-content/uploads/2017/06/Kevin-Bacon-in-Tremors.jpg'}
#body = {'url': 'http://1.bp.blogspot.com/-tSvculYaG4s/VFmUxCD4MJI/AAAAAAAAIAA/7ts97t-ha_c/s1600/movie-pic_002_446_kevin-bacon_wild-things.png'}
#body = {'url': 'https://images.fandango.com/ImageRenderer/400/0/redesign/static/img/default_poster.png/0/images/masterrepository/fandango/113324/frostnixon-pm-kevinbacon-7.jpg'}
#body = {'url': 'http://www3.pictures.zimbio.com/gi/Kevin+Bacon+89W-EsX1Suim.jpg'}

### Executing the API Call

After executing the API call below, you'll see that we get back quite a long JSON response of results. Each of these images contain only Kevin Bacon; but if anyone else had also been present in the photos, their faces would have similar information returned.

In [22]:
try:
    # Execute the REST API call and get the response.
    response = requests.request('POST', uri_base + '/face/v1.0/detect', json=body, data=None, headers=headers, params=params)

    print ('Response:')
    parsed = json.loads(response.text)
    print (json.dumps(parsed, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

Response:
[
  {
    "faceAttributes": {
      "accessories": [],
      "age": 29.1,
      "blur": {
        "blurLevel": "medium",
        "value": 0.43
      },
      "emotion": {
        "anger": 0.094,
        "contempt": 0.001,
        "disgust": 0.0,
        "fear": 0.0,
        "happiness": 0.0,
        "neutral": 0.904,
        "sadness": 0.0,
        "surprise": 0.0
      },
      "exposure": {
        "exposureLevel": "goodExposure",
        "value": 0.53
      },
      "facialHair": {
        "beard": 0.1,
        "moustache": 0.2,
        "sideburns": 0.1
      },
      "gender": "male",
      "glasses": "NoGlasses",
      "hair": {
        "bald": 0.01,
        "hairColor": [
          {
            "color": "brown",
            "confidence": 1.0
          },
          {
            "color": "black",
            "confidence": 0.74
          },
          {
            "color": "blond",
            "confidence": 0.19
          },
          {
            "color": "red",
      

### Analyzing the Results

As you can see in the table of results below, it appears our good friend Mr. Bacon has aged remarkably well over the last few decades. We will discuss a few technical reasons why this error could have occurred during our segment on CNTK and deep learning (pixels in images, face skew, bias in training data, blur levels, etc.).


| Film        | Age (Actual)          | Age (Estimate)  |  Difference |
| ------------- |:-------------:| -----:| -----:|
| _Footloose_   | 26 | 29.1 | +3.1 |
| _Tremors_     | 32 | 31.7 | -0.3 |
| _Wild Things_ | 40 | 39.6 | -0.4 |
| _Frost/Nixon_ | 50 | 44.1 | -5.9 |
| Life          | 59 | 48.2 | -10.8|